In [1]:
import sys
import random
import pickle
import numpy as np
import tensorflow as tf
from tqdm import tqdm

def test(params):
    from sequence_to_sequence import SequenceToSequence
    from data_utils import batch_flow_bucket as batch_flow
    from threadedgenerator import ThreadedGenerator

    x_data, y_data = pickle.load(open('chatbot.pkl', 'rb'))
    ws = pickle.load(open('ws.pkl', 'rb'))

    #训练
    """
    1.n_epoch是训练的轮次数；
    2.理论上来讲训练的轮次数越大，那么训练的精度越高；
    3.如果训练的轮次数特别大，比如1000，那么有可能会发生过拟合的现象，但是是否过拟合也和训练的数据有关
    4.n_epoch越大，训练的时间越长；
    5.办公室电脑用P5000的GPU训练40轮，训练了大概3天，训练2轮，大概一个半小时，
        如果使用CPU来训练的话，速度会特别特别的慢，可能一轮就要几个小时
    """
    n_epoch = 20
    batch_size = 128

    steps = int(len(x_data) / batch_size) + 1

    config = tf.ConfigProto(
        allow_soft_placement=True,#系统自动选择cpu或gpu
        log_device_placement=False#是否打印设备日志
    )

    save_path = './model/s2ss_chatbot.ckpt'

    tf.reset_default_graph()#重置默认图
    with tf.Graph().as_default():
        random.seed(0)
        np.random.seed(0)
        tf.set_random_seed(0)

        with tf.Session(config=config) as sess:
            # 定义模型
            model = SequenceToSequence(
                input_vocab_size=len(ws),
                target_vocab_size=len(ws),
                batch_size=batch_size,
                **params
            )
            init = tf.global_variables_initializer()
            sess.run(init)

            flow = ThreadedGenerator(#把数据放到新的线程里
                batch_flow([x_data, y_data], ws, batch_size, add_end=[False, True]),
                queue_maxsize=30
            )

            for epoch in range(1, n_epoch +1):
                costs = []
                bar = tqdm(range(steps), total=steps,#当前第几步，一共多少步，描述
                           desc='epoch {}, loss=0.000000'.format(epoch))
                for _ in bar:
                    x, xl, y, yl = next(flow)
                    #[[1,2], [3,4]]
                    #[[3,4], [1,2]]
                    x = np.flip(x, axis=1)#数据翻转
                    cost, lr = model.train(sess, x, xl, y, yl, return_lr=True)
                    costs.append(cost)
                    bar.set_description('epoch {} loss={:.6f} lr={:.6f}'.format(
                        epoch,
                        np.mean(costs),
                        lr
                    ))

                model.save(sess, save_path)

    #测试
    tf.reset_default_graph()
    model_pred = SequenceToSequence(
        input_vocab_size=len(ws),
        target_vocab_size=len(ws),
        batch_size=1,
        mode='decode',
        beam_width=12,
        parallel_iterations=1,
        **params
    )

    init = tf.global_variables_initializer()

    with tf.Session(config=config) as sess:
        sess.run(init)
        model_pred.load(sess, save_path)

        bar = batch_flow([x_data, y_data], ws, 1, add_end=False)
        t = 0
        for x, xl, y, yl in bar:
            x = np.flip(x, axis=1)
            pred = model_pred.predict(
                sess,
                np.array(x),
                np.array(xl)
            )
            print(ws.inverse_transform(x[0]))
            print(ws.inverse_transform(y[0]))
            print(ws.inverse_transform(pred[0]))
            t += 1
            if t >= 3:
                break
def main():
    import json
    test(json.load(open('params.json')))

if __name__ == '__main__':
    main()

E:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
seq_dim is deprecated, use seq_axis instead
Instructions for updating:
batch_dim is deprecated, use batch_axis instead


epoch 1 loss=8.686768 lr=0.001000:   0%|          | 5/26093 [00:14<23:58:05,  3.31s/it]

KeyboardInterrupt: 